In [10]:
#import statements
import numpy as np
from astropy.io import ascii
from astropy.table import Table, Column
from astropy import units as u
import h5py

#read in results from MCMC fitting
results=ascii.read('/users/adignan/results_mcmc_filt.csv',format='csv')

#path to MCMC input files
path='/users/adignan/mcmcfiles/'

#initialize empty lists
thermalfrac=[]
thermalfrac_unc=[]

#define equation for calculating free-free emission
def I_ff(nu, T, Aff):
    def gff(nu, T):
        T4 = T/(1.e4*u.K)
        nu9 = nu/(1.*u.GHz)
        gnu = np.log(np.exp(5.960 - (np.sqrt(3.)/np.pi)*np.log(nu9*(T4**(-3./2.)))) + np.exp(1.))
        return gnu   
    
    return Aff*gff(nu,T)/gff(30.*u.GHz,T)

#for loop to calculate thermal fractions and uncertainties
for row in results:
    if row['sourceid']=='NGC0337a':
    
        file=row['sourceid']

        #read in MCMC file
        (nu, fnu, sfnu) = np.genfromtxt(path+file, unpack=True)
        
        print(sfnu)

        #calculate thermal fraction based on best-fit result
        tfrac= I_ff(33*u.GHz,1.e4*u.K,row['A_ff median'])/fnu[2]
        thermalfrac.append(tfrac)

        #calculate uncertainty on thermal fraction

        #load in samples file
        hf = h5py.File('/users/adignan/hdf5/'+str(file)+'-samples_eric.h5', 'r')
        n1 = hf.get('dataset')
        n1=np.array(n1)

        #calculate a few different stats for the samples
        unc=np.std(n1[:,3])
        
        

        #calculate 
        tfrac_unc=np.sqrt( (sfnu[2]/fnu[2])**2 + (unc/tfrac)**2 ) * fnu[2]
        thermalfrac_unc.append(tfrac_unc)

        #print statements for looking at results without saving it to a csv file
        print(r'tfrac: '+"%.3f" % tfrac + ' ± ' +"%.3f" % tfrac_unc)
        
        print(r'tfrac: '+ str(tfrac) + ' ± ' + str(tfrac_unc))

#make astropy table of results
# idscol=Column(results['sourceid'],name='source ID')
# fraccol=Column(thermalfrac,name='thermal fraction')
# unccol=Column(thermalfrac_unc,name='uncertainty')

# tbl1=Table([idscol,fraccol,unccol])
#write results to csv file
# tbl1.write('/users/adignan/thermalfractions_mcmc.csv',format='csv',overwrite=True)

[0.13493866 0.12428348 0.10901988 0.10757986]
tfrac: 0.467 ± 0.293
tfrac: 0.4672513457488573 ± 0.29282706054733837
